# <img src="https://img.buzzfeed.com/buzzfeed-static/static/2017-07/11/14/asset/buzzfeed-prod-fastlane-01/sub-buzz-6725-1499796406-2.png" width="55" height="55"> Hot dog?

[Data](https://www.kaggle.com/datasets/yashvrdnjain/hotdognothotdog/code)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#https://www.geeksforgeeks.org/how-to-iterate-through-images-in-a-folder-python/
import os

# I'm tired of unnecessary warnings 
import warnings
warnings.filterwarnings('ignore')

# Tensorflow will not let me silence it :(

2023-05-26 15:41:15.508095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Data and Build Model

In [2]:
# Define the paths to the directories
train_hot_path = '../train_hotdog'
train_nothot_path = '../train_nothotdog'
test_hot_path = '../test_hotdog'
test_nothot_path = '../test_nothotdog'

# Iterate over the images within the train hotdog folder
train_hotdog_images = []
for file in os.listdir(train_hot_path):
    if file.endswith('.jpg'):
        image_path = os.path.join(train_hot_path, file)
        train_hotdog_images.append(image_path)

# Iterate over the images within the train not hotdog folder
train_nothotdog_images = []
for file in os.listdir(train_nothot_path):
    if file.endswith('.jpg'):
        image_path = os.path.join(train_nothot_path, file)
        train_nothotdog_images.append(image_path)

# Iterate over the images within the test hotdog folder
test_hotdog_images = []
for file in os.listdir(test_hot_path):
    if file.endswith('.jpg'):
        image_path = os.path.join(test_hot_path, file)
        test_hotdog_images.append(image_path)

# Iterate over the images within the test not hotdog folder
test_nothotdog_images = []
for file in os.listdir(test_nothot_path):
    if file.endswith('.jpg'):
        image_path = os.path.join(test_nothot_path, file)
        test_nothotdog_images.append(image_path)

# Create the train_data DataFrame
train_data = pd.DataFrame({
    'image_path': train_hotdog_images + train_nothotdog_images,
    'label': ['hot dog'] * len(train_hotdog_images) + ['not hot dog'] * len(train_nothotdog_images)
})

# Create the test_data DataFrame
test_data = pd.DataFrame({
    'image_path': test_hotdog_images + test_nothotdog_images,
    'label': ['hot dog'] * len(test_hotdog_images) + ['not hot dog'] * len(test_nothotdog_images)
})

# ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_dataframe(
    train_data,
    x_col='image_path',
    y_col='label',
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_dataframe(
    test_data,
    x_col='image_path',
    y_col='label',
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

# Build and train the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
model.add(Flatten())
model.add(Dense(256, activation='relu')) 
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model with EarlyStopping
history = model.fit(train_generator, epochs=100, batch_size=64, validation_data=test_generator, callbacks=[early_stopping])

Found 3000 validated image filenames belonging to 2 classes.
Found 644 validated image filenames belonging to 2 classes.
Epoch 1/100


2023-05-26 15:41:19.231854: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


94/94 [==============================] - ETA: 0s - loss: 0.6849 - accuracy: 0.5497

2023-05-26 15:41:32.037745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


94/94 [==============================] - 15s 149ms/step - loss: 0.6849 - accuracy: 0.5497 - val_loss: 0.6632 - val_accuracy: 0.6056
Epoch 2/100
94/94 [==============================] - 14s 149ms/step - loss: 0.6625 - accuracy: 0.6193 - val_loss: 0.6631 - val_accuracy: 0.6102
Epoch 3/100
94/94 [==============================] - 14s 144ms/step - loss: 0.6371 - accuracy: 0.6423 - val_loss: 0.6483 - val_accuracy: 0.6304
Epoch 4/100
94/94 [==============================] - 14s 150ms/step - loss: 0.6301 - accuracy: 0.6570 - val_loss: 0.6398 - val_accuracy: 0.6242
Epoch 5/100
94/94 [==============================] - 14s 146ms/step - loss: 0.6000 - accuracy: 0.6853 - val_loss: 0.6313 - val_accuracy: 0.6584
Epoch 6/100
94/94 [==============================] - 16s 166ms/step - loss: 0.6006 - accuracy: 0.6893 - val_loss: 0.6204 - val_accuracy: 0.6630
Epoch 7/100
94/94 [==============================] - 15s 155ms/step - loss: 0.5898 - accuracy: 0.6920 - val_loss: 0.6330 - val_accuracy: 0.6242
Epoc

In [3]:
test_loss, test_accuracy = model.evaluate_generator(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

2023-05-26 15:50:27.264486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test Loss: 0.5759133100509644
Test Accuracy: 0.7251552939414978


Not the best, but decent - the high loss is concerning, but I am happy the accuracy is above 70%

In [4]:
# Calculate the baseline accuracy
baseline_accuracy = test_generator.labels.count(0) / len(test_generator.labels)

# Print the baseline accuracy
print('Baseline Accuracy:', baseline_accuracy)

Baseline Accuracy: 0.5


The CNN model works better than the baseline, predicting roughly 20% more accurately than baseline. 

### Save Model

In [5]:
model.save('model.h5') 

# <img src="https://img.buzzfeed.com/buzzfeed-static/static/2017-07/11/5/campaign_images/buzzfeed-prod-fastlane-02/people-are-dying-laughing-at-snapchats-new-hot-do-2-23226-1499766389-1_dblbig.jpg?resize=1200:*" width="300" height="200">